CHAT BOT Customer Sevice dengan NeuralNetwork
Jonathan Kevin 20.K1.0009

1. import dan load data file json

In [7]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

2. Melakukan Preprocess data dengan tokenizing & lematizing

In [8]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [9]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

34 documents
8 classes ['appointment_status', 'check_leave', 'goodbye', 'greetings', 'options', 'search_department', 'search_person_by_id', 'thanks']
69 unique lemmatized words [',', 'ada', 'aji', 'anda', 'apa', 'apakah', 'atas', 'bagaimana', 'bagian', 'bantuan', 'berikan', 'berjumpa', 'bertanya', 'bertemu', 'bisa', 'cukup', 'cuti', 'dapat', 'dengan', 'disini', 'dosen', 'dulu', 'hai', 'halo', 'hari', 'hei', 'informasi', 'informasinya', 'ingin', 'ini', 'jadwal', 'jadwalkan', 'jawabannya', 'jumpa', 'kabarmu', 'kembali', 'ketahui', 'lakukan', 'malam', 'marlon', 'meeting', 'memberikan', 'menanyakan', 'mengetahui', 'menjawab', 'miliki', 'orang', 'pagi', 'pak', 'permisi', 'pertanyaan', 'pertemuan', 'perusahaan', 'petang', 'saja', 'sampai', 'saya', 'sedang', 'selamat', 'sesuatu', 'siang', 'siap', 'sore', 'sudah', 'telah', 'terimakasih', 'tinggal', 'untuk', 'yang']


3. Membuat Training data

In [10]:
# membuat training data kita
training = []
# membuat array kosong untuk output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training, dtype=object)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


4. Membuat model

In [11]:
# Membuat model dengan 3 Layer . Layer Pertama 128 neurons, Layer kedua 64 neurons dan yang ketiga output layer memiliki neuron
# yang sama dengan jumlah intents untuk predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('customers_chatbot_model.h5', hist)

print("model created")

Epoch 1/200
7/7 [==============================] - 1s 5ms/step - loss: 2.0728 - accuracy: 0.1765
Epoch 2/200
7/7 [==============================] - 0s 6ms/step - loss: 1.9949 - accuracy: 0.3529
Epoch 3/200
7/7 [==============================] - 0s 7ms/step - loss: 1.8175 - accuracy: 0.4118
Epoch 4/200
7/7 [==============================] - 0s 9ms/step - loss: 1.7033 - accuracy: 0.5000
Epoch 5/200
7/7 [==============================] - 0s 8ms/step - loss: 1.5934 - accuracy: 0.5000
Epoch 6/200
7/7 [==============================] - 0s 10ms/step - loss: 1.5602 - accuracy: 0.5000
Epoch 7/200
7/7 [==============================] - 0s 9ms/step - loss: 1.5254 - accuracy: 0.4412
Epoch 8/200
7/7 [==============================] - 0s 8ms/step - loss: 1.2153 - accuracy: 0.6471
Epoch 9/200
7/7 [==============================] - 0s 11ms/step - loss: 1.2206 - accuracy: 0.5588
Epoch 10/200
7/7 [==============================] - 0s 7ms/step - loss: 1.1708 - accuracy: 0.6176
Epoch 11/200
7/7 [=========